**Imports and pip installation for the demo**


In [ ]:
!pip install transformers

import torch
import numpy as np

from transformers import AutoTokenizer, AutoModelForTokenClassification

**Loading The Bert Model**


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Andrianos/bert-base-greek-punctuation-prediction-finetuned")
model = AutoModelForTokenClassification.from_pretrained("Andrianos/bert-base-greek-punctuation-prediction-finetuned")

**Samples to Try**
Note: This is an uncased Transformer, capitals are ignored

Writing your own sample to try:
Add all in one string, use words seperated by spaces, do not add punctuation. 

The model will then add the punctuation and you can see the final result in the last slot.

Feel free to add any further procedure to post process text

In [ ]:
# two_sentences = 'προσεκτικά στον δρόμο θα σε περιμένω'
# question = 'τι θα φας για βραδινό'
# multiple_statements = 'κύριε μαυροκέφαλε εσπασέν η αντέννα του ίντερνετ θα πάρω τηλέφωνο την cyta'
news =  'αμεση ήταν η επέμβαση ναυαγοσωστών του ΛΕΜ15 στην παραλία απέναντι από το παλιό εθνικό γήπεδο το απόγευμα του Σαββάτου στο κάλεσμα ανηλίκου λουόμενου για βοήθεια ο νεαρός ενώ κολυμπούσε με φίλο του από τους κυματοθραύστες προς την ακτή λίγο πριν τις 6 λόγω κόπωσης κινδύνευσε με πνιγμό και κάλεσε σε βοήθεια ναυαγοσώστης του ΛΕΜ15 έσπευσε άμεσα και μετέφερε τον νεαρό με ασφάλειά στην ακτή συγχαρητήρια εύγε στον λεβέντη'
orig_tokens = news.split()
sentence = " ".join(orig_tokens)

**Predicting Punctuation using the model**

In [ ]:
tokenized_sentence = tokenizer.encode(sentence)
input_ids = torch.tensor([tokenized_sentence])
with torch.no_grad():
    output = model(input_ids)
predictions = np.argmax(output[0].to('cpu').numpy(), axis=2)
punctuation_points = ['.', ',', ';', '-', ':', '0']

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(punctuation_points[label_idx])
        new_tokens.append(token)

new_tokens = new_tokens[1:-1]
new_labels = new_labels[1:-1]

**Outputting into a single text**

In [ ]:
def to_single_string(tokens, predictions):
    final_string = ""
    for i in range(len(tokens)):
        final_string = final_string + tokens[i]
        if predictions[i] != '0':
            final_string = final_string + predictions[i]
        if predictions[i] != '-':
            final_string = final_string + " "
    return final_string

In [ ]:
print(to_single_string(new_tokens, new_labels))

αμεση ηταν η επεμβαση ναυαγοσωστων του λεμ15 στην παραλια απεναντι απο το παλιο εθνικο γηπεδο το απογευμα του σαββατου. στο καλεσμα ανηλικου λουομενου για βοηθεια ο νεαρος, ενω κολυμπουσε με φιλο του απο τους κυματοθραυστες προς την ακτη λιγο πριν τις 6, λογω κοπωσης, κινδυνευσε με πνιγμο και καλεσε σε βοηθεια. ναυαγοσωστης του λεμ15 εσπευσε αμεσα και μετεφερε τον νεαρο με ασφαλεια στην ακτη. συγχαρητηρια, ευγε στον λεβεντη 
